## Closures

- pokud mám inner funkci a v ní neznámé - python jej nevidí, vidí outer ale inner až poté co je outer spuštěná
- u sdílené proměnné přes dva scopy - tak vytvoří cell se svojí adresou a s adresou na hlavní objekt - hodnotu výslednou (komplikovanější v codu to bude o něco jasnější)
- a částečně to vysvětluje, že se nám nejedná o chain u scopu a jedné hodnoty ale o tom, že nám to směřuje na jeden objekt
- a náš spešl objet tzn. cell který ukazuje na stejnou hodnotu napříč více scopy - tak při ukončení funkce nezaniká a tudíž stále víme jaká je hodnota
- closure je obecně funkce v extended scopu obsahující nonlocal proměnnou 

V podstatě poslední věta výše je zde pěkně znázorněná v obrázku.

- .__code__.co_freevars => introspekce zjistíme jaké jso free variables neboli variables u closure
- .__closure__ => odkazuje na cell a na objekt - v pdostatě adresy týkající se free variables

- closure se vytváří vždycky když probíhá znovu funkce

- tak a jak to dopadne pokud máme dvě closure ale s odkazem na stejný count?

Sdílejí stejný počet, tohle je poměrně zajímavé - obrázek je zde mnohem lepší než psaný výklad.

Tohle máme jako nested closure a to už je docela jízda :D. Takže zde musíme už řádně, pouze názorně v codu uvidíme lépe.

In [65]:
def outer():
    x = "python"
    def inner():
        print(x)
    return inner
# jelikož je x nonlocal variable - vytváří mi funkce inner a x closure

In [67]:
fn = outer()

In [70]:
fn.__code__.co_freevars, fn.__closure__

(('x',), (<cell at 0x000001482B90AFD0: str object at 0x0000014829AC2B30>,))

Takže vidíme, že zde máme free var naše x a ještě adresu našeho cell objektu a str objektu.

In [82]:
def outer():
    x = [1,2,3]
    print(hex(id(x)))
    def inner():
        x = [1,2,3]
        print(hex(id(x)))
    return inner

In [85]:
fn = outer(), fn()

0x1482b90d600
0x1482b90d600


Takže vidíme, že máme stále stejnou adresu.

In [88]:
def outer():
    x = [1,2,3]
    print(hex(id(x)))
    def inner():
        y = x
        print(hex(id(y)))
    return inner

In [96]:
fn = outer()

0x1482b84a700


In [98]:
fn()

0x1482b84a700


In [97]:
fn.__closure__

(<cell at 0x000001482B9339A0: list object at 0x000001482B84A700>,)

Obě ukazuje na cell a obě ukazuje na finální objekt.

In [110]:
def outer():
    count = 0#-cell-0
    def inc():
        nonlocal count #-cell-0
        #sdílený count mezi inc a outer
        count += 1
        return count
    return inc

In [111]:
fn = outer()

In [112]:
fn.__code__.co_freevars #jasné

('count',)

In [113]:
fn.__closure__ #cell směřuje na count a ten na náš objekt - v tomto případě 0

(<cell at 0x000001482B933190: int object at 0x00007FF85C0026F0>,)

In [114]:
fn()

1

In [116]:
fn.__closure__ #změnila se memory adresa našeho objektu - protože jsme zvýšili z 0 na 1
#ale adresa cell je stále stejná

(<cell at 0x000001482B933190: int object at 0x00007FF85C002710>,)

In [119]:
def outer():
    count = 0
    
    def inc1():
        nonlocal count
        count += 1
        return count
    
    def inc2():
        nonlocal count
        count += 1
        return count
    return inc1, inc2
#nyní mám 2x county směřující na stejný objekt
#zapakovaný return

In [120]:
fn1, fn2 = outer() #unpack a volání funkce je prostě luxus :)

In [125]:
fn1.__code__.co_freevars, fn2.__code__.co_freevars,

(('count',), ('count',))

In [126]:
fn1.__closure__, fn2.__closure__

((<cell at 0x000001482B933880: int object at 0x00007FF85C0026F0>,),
 (<cell at 0x000001482B933880: int object at 0x00007FF85C0026F0>,))

Takže jde krásně vidět že jelikož count je ovlivněný objema mají stejnou adresu.

In [127]:
fn1()

1

In [128]:
fn1.__closure__, fn2.__closure__

((<cell at 0x000001482B933880: int object at 0x00007FF85C002710>,),
 (<cell at 0x000001482B933880: int object at 0x00007FF85C002710>,))

A též pokud změním fn1() změni se i fn2() protože jsou propojené s count - oni jsou jen bunky směřující na objekt který je měněn jak z fn1 a fn2

In [136]:
def pow(n):
    def inner(x):
        return x **n
    return inner
#PROČ JE ZDE CLOSURE?
#nonlocal variable n je předáváno výše ve funkci

In [131]:
square = pow(2)

In [132]:
square.__closure__

(<cell at 0x000001482B933820: int object at 0x00007FF85C002730>,)

In [135]:
square.__code__.co_freevars

('n',)

Pokud bych vytvořil pow - jiné n mám jiné closure

In [137]:
def adder(n):
    def inner(x):
        return x + n
    return inner

In [138]:
add_1 = adder(1) 
add_2 = adder(2)
add_3 = adder(3)
#efinuje funkce - které se poté zavolají zase s číslem :)
#ale tentokrát s x ..prostě jim napevno přidávám n - ale abych měl output musím spustit
#add1 apod s x

In [141]:
add_1.__closure__, add_2.__closure__, add_3.__closure__
#máme 3 odlišné closure

((<cell at 0x000001482B9333A0: int object at 0x00007FF85C002710>,),
 (<cell at 0x000001482B933910: int object at 0x00007FF85C002730>,),
 (<cell at 0x000001482B933A30: int object at 0x00007FF85C002750>,))

In [143]:
add_1(10), add_2(10), add_3(10)
#zadáme jim tedy x a máme output ..máme krásní 3 rozdílné closure

(11, 12, 13)

Tak a nyní to zkusíme udělat skrze loop.

In [146]:
adders = []
for n in range(1,4):
    #pomocí lambdy uděláme naší inner funkci
    adders.append(lambda x: x + n )
    #naše n se zde mění podle range
    #closure zde nemáme naše n je global

In [145]:
adders

[<function __main__.<lambda>(x)>,
 <function __main__.<lambda>(x)>,
 <function __main__.<lambda>(x)>]

In [147]:
adders[0].__closure__ #jen ukázka že nemám closer

In [149]:
adders[0](10) #doufal bych že mám první funkci ale nemám

13

Protože naše lambda nemá closure a n se přepisuje, takže si pamatuje pouze poslední n a to je 3..proto máme 13.

In [151]:
adders[0](10), adders[1](10), adders[2](10) #všude stejné

(13, 13, 13)

Nyní to zkusíme napravit a zajistit closure

In [152]:
def create_adders():
    adders = []
    for n in range(1,4):
        adders.append(lambda x: x + n )
    return adders
#jelikžo to máme ve funkci nedostane se mi to z global scopu!
#a tudíž máme closure

In [154]:
adders = create_adders()

In [155]:
adders[0].__closure__

(<cell at 0x000001482B90A0A0: int object at 0x00007FF85C002750>,)

In [156]:
adders[1].__closure__

(<cell at 0x000001482B90A0A0: int object at 0x00007FF85C002750>,)

Máme tedy stejná closure.

In [160]:
adders[0](10), adders[1](10), adders[2](10)

(13, 13, 13)

Takže máme stejný výsledek. Protože všechny naše closures se mění naráz viz jako výše. Proto máme pořád stejný problém, jak jej vyřešit?

In [162]:
def create_adders():
    adders = []
    for n in range(1,4):
        adders.append(lambda x, y=n: x + y )
    return adders

Tak a proč nám tohle funguje? :), protože y není nikde s ničím sdílené a tudíž je pouze v rámci svého scopu a tudíž máme 4x různé y.

In [163]:
adders = create_adders()

In [164]:
adders[0](10), adders[1](10), adders[2](10)

(11, 12, 13)

- adders.append(lambda x: x + n )
- adders.append(lambda x, y=n: x + y )
- A PROČ PROTOŽE y není freevariable a obejdu closure a hlavně naše hodnota není viděná a načte se pythnem až když lambda proběhne

In [165]:
adders[0].__closure__, adders[0].__code__.co_freevars

(None, ())